In [ ]:
import pandas as pd
import numpy as np
import gc
import os

# ==============================================================================
# [스크립트 1] 1차 보간 파일 생성기 (train/test 공용) - (수정본)
# ==============================================================================

# 1. 설정 변수 정의
# ------------------------------------------------------------------------------
# ★★★ 이 부분을 수정하여 'train'용과 'test'용 파일을 생성합니다 ★★★

# 예: train.csv를 처리할 때
INPUT_FILE = 'train.csv'
OUTPUT_FILE = '1차_train.feather'
IS_TRAIN_MODE = True # nins (TARGET) 열이 있는지 여부

# 예: test.csv를 처리할 때
# INPUT_FILE = 'test.csv'
# OUTPUT_FILE = '1차_test_보간.feather'
# IS_TRAIN_MODE = False
# ------------------------------------------------------------------------------

CHUNK_SIZE = 7000000
TARGET = 'nins'

# 모델 학습에 사용할 최종 특성 리스트 (수정됨)
# wind_gust_spd 추가
# wind_dir/spd 대신 U/V 벡터 추가
FEATURES = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis',
    # 'wind_dir_b', 'wind_spd_b', # 벡터로 대체됨
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a',
    # 'wind_dir_a', 'wind_spd_a', # 벡터로 대체됨
    'coord1', 'coord2', 'time',
    'pv_id',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'wind_gust_spd', # (재-추가) 돌풍 피처
    # (신규) 벡터 성분 피처
    'U_a', 'V_a', 'U_b', 'V_b'
]

# 원본 CSV에서 읽어올 피처 (U/V 변환에 필요)
# U/V로 변환 후 버릴 컬럼도 여기에 포함되어야 함
BASE_FEATURES = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis', 'wind_dir_b', 'wind_spd_b', # 원본
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a', 'wind_dir_a', 'wind_spd_a', # 원본
    'coord1', 'coord2', 'time',
    'pv_id',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'wind_gust_spd' # (재-추가)
]

# 2. 데이터 처리 및 피처 엔지니어링 함수
def process_data_chunk(chunk_df):
    chunk_df['pv_id'] = chunk_df['pv_id'].astype('category')
    
    for col in chunk_df.select_dtypes(include=['float64']).columns:
        chunk_df[col] = chunk_df[col].astype('float32')
    for col in chunk_df.select_dtypes(include=['int64']).columns:
        chunk_df[col] = chunk_df[col].astype('int32')

    chunk_df['time'] = pd.to_datetime(chunk_df['time'], errors='coerce')
    
    # 사용할 칼럼 리스트 준비
    final_features = [f for f in BASE_FEATURES if f in chunk_df.columns]
    
    cols_to_return = final_features
    if IS_TRAIN_MODE and TARGET in chunk_df.columns:
        cols_to_return = final_features + [TARGET]
    elif not IS_TRAIN_MODE:
        cols_to_return = final_features

    return chunk_df[cols_to_return]

# 3. 대용량 데이터 로딩 및 전처리 실행
print(f"파일 로딩 시작: {INPUT_FILE}")
processed_chunks = []
for chunk in pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE, engine='python', sep=','):
    processed_chunk = process_data_chunk(chunk)
    processed_chunks.append(processed_chunk)

df = pd.concat(processed_chunks, ignore_index=True)
del processed_chunks
gc.collect()
print("기본 처리 및 통합 완료.")

# 4. ★★★ (신규) 바람 벡터 변환 ★★★
# ------------------------------------------------------------------------------
print("풍향/풍속을 U, V 벡터 성분으로 변환합니다...")

def convert_wind_to_vectors(df, prefix):
    dir_col = f'wind_dir_{prefix}'
    spd_col = f'wind_spd_{prefix}'
    u_col = f'U_{prefix}'
    v_col = f'V_{prefix}'
    
    if dir_col in df.columns and spd_col in df.columns:
        # 라디안 변환
        rad = df[dir_col] * np.pi / 180.0
        # U, V 계산
        df[u_col] = df[spd_col] * np.cos(rad)
        df[v_col] = df[spd_col] * np.sin(rad)
    return df

df = convert_wind_to_vectors(df, 'a')
df = convert_wind_to_vectors(df, 'b')

# 원본 풍향/풍속 컬럼은 이제 보간 대상에서 제외 (FEATURES 리스트에서 이미 빠져있음)
print("U/V 벡터 변환 완료.")
# ------------------------------------------------------------------------------


# 5. 하이브리드 보간 (수정된 전략)
# ------------------------------------------------------------------------------
print("수정된 하이브리드 보간법(Linear, Ffill)을 시작합니다...")
df.sort_values(by=['pv_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.loc[:, ~df.columns.duplicated(keep='first')]

# 1. 보간할 칼럼 그룹 정의 (수정됨)

# (1) 연속형 변수 -> 'linear' 보간##나머지는 time, pv_id, nise, energy, type, coord1,coord2#7 ground_press', 'wind_gust_spd', 'wind_chill_temp', 'rel_hum'#4(보간필요없거나 리스트에서 제거한 필요없는값)
linear_cols = [
    'appr_temp', 'dew_point', 'pressure', 'temp_b',
    'temp_a', 'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'humidity', 'wind_gust_spd',
    # 신규 벡터 컬럼
    'U_a', 'V_a', 'U_b', 'V_b'
]

# (2) 상태형 변수 -> 'ffill' (Forward Fill) 보간
ffill_cols = [
    'ceiling', 'cloud_b', 'precip_1h', 'uv_idx', 'vis',
    'cloud_a', 'rain', 'snow'
]

# 데이터프레임에 실제 존재하는 컬럼들만 필터링
linear_cols = [col for col in linear_cols if col in df.columns]
ffill_cols = [col for col in ffill_cols if col in df.columns]

# (안전장치) 보간 대상 칼럼들을 numeric으로 강제 변환
print("보간 대상 칼럼들을 numeric으로 강제 변환합니다 (errors='coerce')...")
all_interp_cols = linear_cols + ffill_cols
for col in all_interp_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. 그룹별 보간 실행
g = df.groupby('pv_id')

# (1단계) Linear 보간ㅋ
print(f"{len(linear_cols)}개 열에 대해 'linear' 보간을 수행합니다...")
df[linear_cols] = g[linear_cols].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)

# (2단계) Ffill 보간 (ffill 후 bfill로 맨 앞 결측치까지 채움) 걍 순서 바꾸자
print(f"{len(ffill_cols)}개 열에 대해 'bfill' 보간..")
df[ffill_cols] = g[ffill_cols].transform(
    lambda x: x.bfill().ffill()
)

# (3단계) 보간 후 남은 NaN을 0으로 임시 처리
# (대부분 bfill로 채워지지만, 발전소 데이터 전체가 비어있는 극단적 경우 대비)
print("보간 후 남은 NaN을 0으로 임시 처리합니다...")
df[all_interp_cols] = df[all_interp_cols].fillna(0)

print("하이브리드 보간 완료.")
# ------------------------------------------------------------------------------

# 6. 1차 파일 저장 (체크포인트)
# 최종 피처 리스트만 선택하여 저장
final_feature_list = [f for f in FEATURES if f in df.columns]
if IS_TRAIN_MODE and TARGET in df.columns:
    final_feature_list += [TARGET]

df_to_save = df[final_feature_list]

print(f"\n보간이 완료된 1차 데이터를 '{OUTPUT_FILE}' 파일로 저장합니다...")
df_to_save.to_feather(OUTPUT_FILE)
print("1차 파일 저장 완료.")

파일 로딩 시작: train.csv
기본 처리 및 통합 완료.
풍향/풍속을 U, V 벡터 성분으로 변환합니다...
U/V 벡터 변환 완료.
수정된 하이브리드 보간법(Linear, Ffill)을 시작합니다...
보간 대상 칼럼들을 numeric으로 강제 변환합니다 (errors='coerce')...
15개 열에 대해 'linear' 보간을 수행합니다...
8개 열에 대해 'ffill'/'bfill' 보간을 수행합니다...
보간 후 남은 NaN을 0으로 임시 처리합니다...
하이브리드 보간 완료.

보간이 완료된 1차 데이터를 '1차_train.feather' 파일로 저장합니다...
1차 파일 저장 완료.


In [ ]:
#보간보다 값을 유지하는게 모델한테 더 좋을 수 있다고함 테스트해볼만한 가치 있음

In [ ]:
import pandas as pd
import numpy as np
import gc
import os

# ==============================================================================
# [스크립트 1] 1차 보간 파일 생성기 (bfill 우선 수정본)
# ==============================================================================

# 1. 설정 변수 정의
# ------------------------------------------------------------------------------
# ★★★ 이 부분을 수정하여 'train'용과 'test'용 파일을 생성합니다 ★★★

# 예: train.csv를 처리할 때
INPUT_FILE = 'train.csv'
OUTPUT_FILE = '1차_train.feather'
IS_TRAIN_MODE = True # nins (TARGET) 열이 있는지 여부

# 예: test.csv를 처리할 때
# INPUT_FILE = 'test.csv'
# OUTPUT_FILE = '1차_test_보간.feather'
# IS_TRAIN_MODE = False
# ------------------------------------------------------------------------------

CHUNK_SIZE = 7000000
TARGET = 'nins'

# 모델 학습에 사용할 최종 특성 리스트
FEATURES = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis',
    # 'wind_dir_b', 'wind_spd_b', # 벡터로 대체됨
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a',
    # 'wind_dir_a', 'wind_spd_a', # 벡터로 대체됨
    'coord1', 'coord2', 'time',
    'pv_id',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'wind_gust_spd', # (재-추가) 돌풍 피처
    # (신규) 벡터 성분 피처
    'U_a', 'V_a', 'U_b', 'V_b'
]

# 원본 CSV에서 읽어올 피처 (U/V 변환에 필요)
BASE_FEATURES = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis', 'wind_dir_b', 'wind_spd_b', # 원본
    'cloud_a', 'humidity', 'rain', 'snow', 'temp_a', 'wind_dir_a', 'wind_spd_a', # 원본
    'coord1', 'coord2', 'time',
    'pv_id',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'wind_gust_spd' # (재-추가)
]

# 2. 데이터 처리 및 피처 엔지니어링 함수
def process_data_chunk(chunk_df):
    chunk_df['pv_id'] = chunk_df['pv_id'].astype('category')
    
    for col in chunk_df.select_dtypes(include=['float64']).columns:
        chunk_df[col] = chunk_df[col].astype('float32')
    for col in chunk_df.select_dtypes(include=['int64']).columns:
        chunk_df[col] = chunk_df[col].astype('int32')

    chunk_df['time'] = pd.to_datetime(chunk_df['time'], errors='coerce')
    
    # 사용할 칼럼 리스트 준비
    final_features = [f for f in BASE_FEATURES if f in chunk_df.columns]
    
    cols_to_return = final_features
    if IS_TRAIN_MODE and TARGET in chunk_df.columns:
        cols_to_return = final_features + [TARGET]
    elif not IS_TRAIN_MODE:
        cols_to_return = final_features

    return chunk_df[cols_to_return]

# 3. 대용량 데이터 로딩 및 전처리 실행
print(f"파일 로딩 시작: {INPUT_FILE}")
processed_chunks = []
for chunk in pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE, engine='python', sep=','):
    processed_chunk = process_data_chunk(chunk)
    processed_chunks.append(processed_chunk)

df = pd.concat(processed_chunks, ignore_index=True)
del processed_chunks
gc.collect()
print("기본 처리 및 통합 완료.")

# 4. ★★★ (신규) 바람 벡터 변환 ★★★
# ------------------------------------------------------------------------------
print("풍향/풍속을 U, V 벡터 성분으로 변환합니다...")

def convert_wind_to_vectors(df, prefix):
    dir_col = f'wind_dir_{prefix}'
    spd_col = f'wind_spd_{prefix}'
    u_col = f'U_{prefix}'
    v_col = f'V_{prefix}'
    
    if dir_col in df.columns and spd_col in df.columns:
        # 라디안 변환
        rad = df[dir_col] * np.pi / 180.0
        # U, V 계산
        df[u_col] = df[spd_col] * np.cos(rad)
        df[v_col] = df[spd_col] * np.sin(rad)
    return df

df = convert_wind_to_vectors(df, 'a')
df = convert_wind_to_vectors(df, 'b')

print("U/V 벡터 변환 완료.")
# ------------------------------------------------------------------------------


# 5. 하이브리드 보간 (수정된 전략: bfill 우선 적용)
# ------------------------------------------------------------------------------
print("수정된 하이브리드 보간법(Bfill -> Ffill)을 시작합니다...")
df.sort_values(by=['pv_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.loc[:, ~df.columns.duplicated(keep='first')]

# 1. 보간할 칼럼 그룹 정의 (linear 사용 안 함)

linear_cols = [] # linear 사용 안 함

bfill_cols = [
    # (상태형 변수)
    'ceiling', 'cloud_b', 'precip_1h', 'uv_idx', 'vis',
    'cloud_a', 'rain', 'snow',
    # (1시간 단위 변수들)
    'appr_temp', 'dew_point', 'pressure', 'temp_b',
    'temp_a', 'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'humidity', 'wind_gust_spd',
    # (신규 벡터 컬럼)
    'U_a', 'V_a', 'U_b', 'V_b'
]

# 데이터프레임에 실제 존재하는 컬럼들만 필터링
linear_cols = [col for col in linear_cols if col in df.columns]
bfill_cols = [col for col in bfill_cols if col in df.columns]

# (안전장치) 보간 대상 칼럼들을 numeric으로 강제 변환
print("보간 대상 칼럼들을 numeric으로 강제 변환합니다 (errors='coerce')...")
all_interp_cols = linear_cols + bfill_cols
for col in all_interp_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. 그룹별 보간 실행
g = df.groupby('pv_id')

# (1단계) Linear 보간 (수행할 컬럼 없음)
# (생략)

# (2단계) Bfill 보간 (bfill 후 ffill로 맨 마지막 결측치까지 채움)
print(f"{len(bfill_cols)}개 열에 대해 'bfill'/'ffill' 보간을 수행합니다...")
df[bfill_cols] = g[bfill_cols].transform(
    lambda x: x.bfill().ffill()  # ★★★ 이 부분이 요청하신 bfill 우선 적용입니다 ★★★
)

# (3단계) 보간 후 남은 NaN을 0으로 임시 처리
# (bfill().ffill()로도 채워지지 않는, 그룹 전체가 NaN인 극단적 경우 대비)
print("보간 후 남은 NaN을 0으로 임시 처리합니다...")
df[all_interp_cols] = df[all_interp_cols].fillna(0)

print("하이브리드 보간 완료.")
# ------------------------------------------------------------------------------

# 6. 1차 파일 저장 (체크포인트)
# 최종 피처 리스트만 선택하여 저장
final_feature_list = [f for f in FEATURES if f in df.columns]
if IS_TRAIN_MODE and TARGET in df.columns:
    final_feature_list += [TARGET]

df_to_save = df[final_feature_list]

print(f"\n보간이 완료된 1차 데이터를 '{OUTPUT_FILE}' 파일로 저장합니다...")
df_to_save.to_feather(OUTPUT_FILE)
print("1차 파일 저장 완료.")

파일 로딩 시작: train.csv
기본 처리 및 통합 완료.
풍향/풍속을 U, V 벡터 성분으로 변환합니다...
U/V 벡터 변환 완료.
수정된 하이브리드 보간법(Bfill -> Ffill)을 시작합니다...
보간 대상 칼럼들을 numeric으로 강제 변환합니다 (errors='coerce')...
23개 열에 대해 'bfill'/'ffill' 보간을 수행합니다...
하이브리드 보간 완료.

보간이 완료된 1차 데이터를 '1차_train.feather' 파일로 저장합니다...
1차 파일 저장 완료.
